In [1]:
import pandas as pd
df_guardians = pd.read_csv('../CSVs/Guardians_Full_Dataset.csv')

In [3]:
# Define the hitter ID-to-name mapping
hitter_mapping = {
    680757: "Steven Kwan",
    665926: "Andres Gimenez",
    608070: "Jose Ramirez",
    647304: "Josh Naylor",
    657041: "Lane Thomas",
    677587: "Bryan Rocchio",
    666310: "Bo Naylor",
    681807: "David Fry",
    686823: "Will Brennan",
    671289: "Tyler Freeman",
    682177: "Daniel Schneemann",
    678877: "Jhonkensy Noel",
    672356: "Gabriel Arias",
    682657: "Angel Martinez",
    700932: "Kyle Manzardo",
    595978: "Austin Hedges"
}


In [5]:
# Replace hitter IDs with names
df_guardians['batter'] = df_guardians['batter'].map(hitter_mapping)

# Drop rows where the batter is not in the mapping (i.e., players with minimal data)
df_guardians = df_guardians.dropna(subset=['batter'])


In [7]:
# Display unique values in the batter column
df_guardians['batter'].unique()


array(['Bryan Rocchio', 'Jhonkensy Noel', 'Bo Naylor', 'Andres Gimenez',
       'Lane Thomas', 'Josh Naylor', 'Jose Ramirez', 'Kyle Manzardo',
       'Will Brennan', 'Angel Martinez', 'David Fry', 'Steven Kwan',
       'Daniel Schneemann', 'Austin Hedges', 'Tyler Freeman',
       'Gabriel Arias'], dtype=object)

In [9]:
# Save the cleaned Guardians dataset with hitter names
df_guardians.to_csv("Guardians_With_100_ABs.csv", index=False)


In [9]:
# Step 1: Identify the first pitch of each at-bat
df_first_pitch = df_guardians[df_guardians['pitch_number'] == 1]  # Explicitly take first pitch

# Debugging: Check total at-bats (should be ~5,947)
print(f"Total At-Bats Identified: {len(df_first_pitch)}")  

# Step 2: Filter for swings on that first pitch
swing_terms = ['swinging_strike', 'foul', 'hit_into_play']
df_first_pitch_swing = df_first_pitch[df_first_pitch['description'].str.contains('|'.join(swing_terms), na=False)]

# Debugging: Check first-pitch swings count
print(f"First-Pitch Swings Count: {len(df_first_pitch_swing)}")

# Step 3: Rename 'events' in df_first_pitch_swing to avoid conflicts
df_first_pitch_swing = df_first_pitch_swing.rename(columns={'events': 'first_pitch_events'})

# Step 4: Extract final at-bat outcomes (last pitch of each at-bat)
df_final_results = df_guardians.sort_values(by=['game_pk', 'at_bat_number'])

# Ensure 'events' column is not missing before proceeding
if 'events' not in df_final_results.columns:
    raise KeyError("Column 'events' is missing from df_final_results!")

# Drop duplicates to get only the last pitch of each at-bat
df_final_results = df_final_results.dropna(subset=['events'])  # Ensure we don't lose important rows
df_final_results = df_final_results.drop_duplicates(subset=['game_pk', 'at_bat_number'], keep='last')

# Keep only necessary columns
df_final_results = df_final_results[['game_pk', 'at_bat_number', 'events']]

# Debugging: Confirm final at-bat results are correct
print(f"Total Final At-Bat Results: {len(df_final_results)}")

# Step 5: Merge ONLY first-pitch swings with final at-bat results
df_merged = df_first_pitch_swing.merge(df_final_results, on=['game_pk', 'at_bat_number'], how='left')

# Debugging: Check merged dataset row count
print(f"Total Merged Rows (should match first-pitch swings count): {len(df_merged)}")

# Step 6: Ensure correct 'events' column (final at-bat result)
if 'events_y' in df_merged.columns:
    df_merged = df_merged.rename(columns={'events_y': 'events'}).drop(columns=['events_x'], errors='ignore')

# Step 7: Keep only relevant columns
columns_to_keep = ['game_pk', 'at_bat_number', 'batter', 'description', 'des', 'first_pitch_events', 'events']
df_cleaned = df_merged[columns_to_keep]

# Step 8: Save the final cleaned dataset
df_cleaned.to_csv("Guardians_First_Pitch_Swings_Cleaned.csv", index=False)

print("Final cleaned dataset saved as 'Guardians_First_Pitch_Swings_Cleaned.csv'")


Total At-Bats Identified: 5936
First-Pitch Swings Count: 1818
Total Final At-Bat Results: 5932
Total Merged Rows (should match first-pitch swings count): 1818
Final cleaned dataset saved as 'Guardians_First_Pitch_Swings_Cleaned.csv'
